# zib gereed maken voor mapping

* inlezen en layout consistent maken met XtEHR
* opslaan voor mapping - zie mapping Notebook

## imports

In [1]:
import pandas as pd
import os

## toon de zibs in folder (hier: zib2024)

In [3]:
base = 'zibs2024'
folder = os.path.join(base, 'downloads')
files = os.listdir(folder)
[(i, file) for i, file in enumerate(files)]

[(0, 'nl.zorg.AdministrationAgreement-v3.0(2024EN).xlsx'),
 (1, 'Nl.zorg.Alert-v5.0(2024EN).xlsx'),
 (2, 'Nl.zorg.Condition-v1.1(2024EN).xlsx'),
 (3, 'Nl.zorg.Diagnosis-v2.0(2024EN).xlsx'),
 (4, 'nl.zorg.DispenseRequest-v4.0(2024EN).xlsx'),
 (5, 'Nl.zorg.FunctionalOrMentalStatus-v4.0.1(2024EN).xlsx'),
 (6, 'Nl.zorg.HealthcareProvider-v3.6(2024EN).xlsx'),
 (7, 'Nl.zorg.HealthProfessional-v4.0(2024EN).xlsx'),
 (8, 'Nl.zorg.MedicalDevice-v5.0(2024EN).xlsx'),
 (9, 'nl.zorg.MedicationDispense-v3.0.1(2024EN).xlsx'),
 (10, 'Nl.zorg.NursingIntervention-v5.0(2024EN).xlsx'),
 (11, 'Nl.zorg.part.AddressInformation-v1.2(2024EN).xlsx'),
 (12, 'Nl.zorg.part.AnatomicalLocation-v1.0.4(2024EN).xlsx'),
 (13, 'Nl.zorg.part.ContactInformation-v1.3.1(2024EN).xlsx'),
 (14, 'Nl.zorg.part.InstructionsForUse-v3.1(2024EN).xlsx'),
 (15, 'Nl.zorg.part.NameInformation-v1.2(2024EN).xlsx'),
 (16, 'Nl.zorg.part.PharmaceuticalProduct-v2.3(2024EN).xlsx'),
 (17, 'Nl.zorg.Patient-v4.3(2024EN).xlsx'),
 (18, 'Nl.zorg.Proce

## kies een zib

In [4]:
file = files[5]
print(file)
url = os.path.join(folder, file)
zibdf = pd.read_excel(url, sheet_name='Data', header=1)
zibdf

Nl.zorg.FunctionalOrMentalStatus-v4.0.1(2024EN).xlsx


C:\Users\engel_nict1\AppData\Roaming\Python\Python313\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Unnamed: 0,Concept,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Alias,Type,Card.,Stereotype,Id,Definition,DefinitionCode,Reference,Constraints
0,NaN,FunctionalOrMentalStatus,NaN,NaN,NaN,NaN,NaN,NL: FunctioneleOfMentaleStatus,NaN,NaN,rootconcept,NL-CM:4.26.1,Root concept of FunctionalOrMentalStatus. This...,NaN,NaN,NaN
1,NaN,NaN,StatusName,NaN,NaN,NaN,NaN,NL: StatusNaam,CD,1,data,NL-CM:4.26.2,Coded description of the functional or mental ...,NaN,StatusNameICFCodelist,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,StatusNameSnomedCodelist,NaN
3,NaN,NaN,StatusValue,NaN,NaN,NaN,NaN,NL: StatusWaarde,CO,0..1,data,NL-CM:4.26.3,The extent to which the patient is limited by ...,NaN,StatusValueICFCodelist,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,StatusValueSnomedCodelist,NaN
5,NaN,NaN,StatusDate,NaN,NaN,NaN,NaN,NL: StatusDatum,TS,0..1,data,NL-CM:4.26.6,Date on which the status is or was determined....,NaN,NaN,NaN
6,NaN,NaN,Comment,NaN,NaN,NaN,NaN,NL: Toelichting,ST,0..1,data,NL-CM:4.26.4,Explanatory comments as a description of the f...,LOINC: 48767-8 Annotation comment [Interpretat...,NaN,NaN
7,NaN,NaN,MedicalDevice:MedicalDevice::MedicalDevice::Me...,NaN,NaN,NaN,NaN,NL: Hulpmiddel::MedischHulpmiddel,NaN,0..*,"data,reference",NL-CM:4.26.5,Medical aid the patient has or will receive to...,NaN,This is a reference to the rootconcept of info...,NaN


## opschonen zib

* lowercase kolomnamen
* paar labels laten vervallen
* name en path kolom toevoegen
* name vullen (naam attribuut)
* kolommen opschonen tot benodigde

Handmatig:
* path vullen (als in logical model, dus pad.pad2.pad3.naam etc.)

In [5]:
zibdf.columns = [col.lower() for col in zibdf.columns]
zibdf = zibdf.drop(labels=['unnamed: 0', 'reference'], axis=1).dropna(how='all')
zibdf.insert(0, 'name', None)
zibdf.insert(0, 'path', None)
zibdf.fillna('')
concepts = []
for i, row in zibdf.iterrows():
    # Loop thru columns 2 tot 8 (Concept and 'Unnamed' ones)
    for j in range(2, 6):
        # print(j, row.iloc[j])
        if not pd.isna(row.iloc[j]):
            # Remove last element if next column is empty
            if len(concepts) > j -2:
                concepts.pop()
            try:
                concepts[j - 2] = row.iloc[j]
            except IndexError:
                concepts.append(row.iloc[j])
    # print(i, concepts)
    zibdf.at[i, 'path'] = '.'.join(concepts)
    zibdf.at[i, 'name'] = concepts[-1]
zibdf = zibdf[['name', 'path', 'alias', 'type', 'card.', 'stereotype', 'id', 'definition', 'definitioncode']].fillna('')
zibdf

,name,path,alias,type,card.,stereotype,id,definition,definitioncode
0,FunctionalOrMentalStatus,FunctionalOrMentalStatus,NL: FunctioneleOfMentaleStatus,,,rootconcept,NL-CM:4.26.1,Root concept of FunctionalOrMentalStatus. This...,
1,StatusName,FunctionalOrMentalStatus.StatusName,NL: StatusNaam,CD,1,data,NL-CM:4.26.2,Coded description of the functional or mental ...,
3,StatusValue,FunctionalOrMentalStatus.StatusValue,NL: StatusWaarde,CO,0..1,data,NL-CM:4.26.3,The extent to which the patient is limited by ...,
5,StatusDate,FunctionalOrMentalStatus.StatusDate,NL: StatusDatum,TS,0..1,data,NL-CM:4.26.6,Date on which the status is or was determined....,
6,Comment,FunctionalOrMentalStatus.Comment,NL: Toelichting,ST,0..1,data,NL-CM:4.26.4,Explanatory comments as a description of the f...,LOINC: 48767-8 Annotation comment [Interpretat...
7,MedicalDevice:MedicalDevice::MedicalDevice::Me...,FunctionalOrMentalStatus.MedicalDevice:Medical...,NL: Hulpmiddel::MedischHulpmiddel,,0..*,"data,reference",NL-CM:4.26.5,Medical aid the patient has or will receive to...,


## Excel opslaan

In [6]:
url = os.path.join(base, 'mappable', file)
zibdf.to_excel(url, index=False)